Seminar 5

In [1]:
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
import csv

We say structures are similar if Tanimoto index is > 0.8

In [2]:
# load your own set, and DrugBank :)
with open('/Users/emafialova/cdd2025/data/chembl_mtor_ic50.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=";")
    mtor_ligands = [Chem.MolFromSmiles(m['Smiles']) for m in reader]

suppl = Chem.SDMolSupplier('/Users/emafialova/cdd2025/data/drugbank.sdf')
drugs = [m for m in suppl if m]

[12:29:03] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 28 ignored.
[12:29:03] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 28 ignored.
[12:29:03] Can't kekulize mol.  Unkekulized atoms: 1 2 3 5 6 7 8 9 10
[12:29:03] ERROR: Could not sanitize molecule ending on line 250947
[12:29:03] ERROR: Can't kekulize mol.  Unkekulized atoms: 1 2 3 5 6 7 8 9 10
[12:29:03] Explicit valence for atom # 17 O, 2, is greater than permitted
[12:29:03] ERROR: Could not sanitize molecule ending on line 258130
[12:29:03] ERROR: Explicit valence for atom # 17 O, 2, is greater than permitted
[12:29:03] Can't kekulize mol.  Unkekulized atoms: 57 58 59 60 61 62 63 64 65
[12:29:03] ERROR: Could not sanitize molecule ending on line 261581
[12:29:03] ERROR: Can't kekulize mol.  Unkekulized atoms: 57 58 59 60 61 62 63 64 65
[12:29:03] Can't kekulize mol.  Unkekulized atoms: 0 1 2 6 7 8 9 10 11 12 13 14 15 16 17
[12:29:03] ERROR: Could not 

In [3]:
from rdkit.Chem import AllChem
fpgen = AllChem.GetMorganGenerator(radius=2)
m1 = Chem.MolFromSmiles('Cc1ccccc1')
fp1 = fpgen.GetSparseCountFingerprint(m1)
fp1

In [4]:
fp1.GetNonzeroElements()

{98513984: 3,
 422715066: 1,
 908339072: 1,
 951226070: 2,
 2246728737: 1,
 2763854213: 1,
 3207567135: 1,
 3217380708: 1,
 3218693969: 5,
 3999906991: 2,
 4244175903: 2}

In [5]:
fp1.ToBinary()

b"\x01\x00\x00\x00\x08\x00\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff\x0b\x00\x00\x00\x00\x00\x00\x00@4\xdf\x05\x00\x00\x00\x00\x03\x00\x00\x00\xba\x1e2\x19\x00\x00\x00\x00\x01\x00\x00\x00\x80'$6\x00\x00\x00\x00\x01\x00\x00\x00\xd6\x8e\xb28\x00\x00\x00\x00\x02\x00\x00\x00!\\\xea\x85\x00\x00\x00\x00\x01\x00\x00\x00\x85\x11\xbd\xa4\x00\x00\x00\x00\x01\x00\x00\x00\x1f\x97/\xbf\x00\x00\x00\x00\x01\x00\x00\x00dU\xc5\xbf\x00\x00\x00\x00\x01\x00\x00\x00Q_\xd9\xbf\x00\x00\x00\x00\x05\x00\x00\x00\xaf\xbci\xee\x00\x00\x00\x00\x02\x00\x00\x00\x1f\xfc\xf8\xfc\x00\x00\x00\x00\x02\x00\x00\x00"

In [10]:
fp2 = fpgen.GetFingerprint(m1)
list(fp2.GetOnBits())

[389, 1055, 1057, 1088, 1199, 1380, 1722, 1750, 1823, 1873, 1920]

Looking for molecules from mtor that are similar to molecules in drugbank = need to know which ones.

In [31]:
from rdkit import DataStructs
def similarity_search(fp, fps, threshold = 0.8):
    similar = []
    for index, i in enumerate(fps):
        similarity = DataStructs.TanimotoSimilarity(fp, i)
        if similarity >= threshold:
            similar.append(index)
    return similar

In [12]:
drug_fps = [fpgen.GetFingerprint(drug) for drug in drugs]
mtor_fps = [fpgen.GetFingerprint(drug) for drug in mtor_ligands]

In [32]:
result = [(index, similarity_search(mtor, drug_fps)) for index, mtor in enumerate(mtor_fps)]

In [34]:
len([(m,ds) for m,ds in result if len(ds)])

56

In [35]:
relevant = [(m,ds) for m,ds in result if len(ds)]
print(relevant)

[(17, [1099]), (23, [760, 1377, 4580]), (36, [5903]), (77, [760, 1377]), (145, [287, 6682]), (205, [5905]), (211, [5904]), (218, [5214]), (317, [760, 1377, 4580]), (338, [1700]), (373, [747]), (379, [760, 1377, 4580]), (496, [1130]), (517, [4580]), (540, [5910]), (551, [747]), (566, [5468]), (888, [5904]), (889, [5904]), (906, [6143]), (954, [6143]), (1002, [287, 6682]), (1045, [2344]), (1057, [760]), (1060, [2344]), (1067, [760, 1377, 4580]), (1095, [1141, 5860]), (1134, [5121]), (1218, [92]), (1274, [1575]), (1312, [2344]), (1322, [760, 1377, 4580]), (1391, [2344]), (1406, [760, 1377, 4580]), (1416, [760, 1377, 4580]), (1520, [4580]), (1542, [760, 1377, 4580]), (1550, [4580]), (1600, [2344]), (1629, [1700]), (1712, [2344]), (1730, [2344]), (1777, [5904]), (1783, [208]), (1818, [5905]), (1825, [5904]), (1826, [5903]), (1827, [5904]), (1939, [1130]), (2024, [760, 1377, 4580]), (2796, [2344]), (2807, [1756]), (3046, [5903]), (3047, [5214]), (3200, [2344]), (3478, [6804])]


In [37]:
for i in relevant[:4]:
    Draw.MolsToGridImage(mtor_ligands[i[0]])

TypeError: object of type 'Mol' has no len()